In [1]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.13.7
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

#  Génération de Grilles de Mots-Croisés par Programmation par Contraintes

 **Groupe :** Baptiste Villeneuve / Lucas Juanico / Killian Maurin
 
 **Date :** 11/04/2025

 # 1. Introduction

 Ce notebook détaille la construction d'un générateur automatique de grilles de mots-croisés en utilisant la Programmation par Contraintes (PPC). L'objectif est de remplir une grille prédéfinie (cases noires/blanches) avec des mots issus d'un dictionnaire, en s'assurant que les lettres aux intersections sont cohérentes.

 Nous utilisons la bibliothèque **OR-Tools** de Google, et plus spécifiquement son solveur **CP-SAT**, qui est particulièrement adapté à ce type de problème combinatoire.

# 2. Définition du Problème

 **Étant donné :**
 1.  Une structure de grille (matrice de caractères indiquant les cases noires '#' et les cases blanches '.').
 2.  Un dictionnaire de mots valides (`dictionnaire.txt`).
 3.  Une longueur minimale pour les mots acceptés dans la grille.

 **Trouver :**
 Une assignation de mots du dictionnaire à chaque séquence horizontale ou verticale de cases blanches (appelée "emplacement" ou "slot") de longueur au moins minimale, telle que :
 *   Chaque emplacement est rempli par un mot de la bonne longueur issu du dictionnaire.
 *   Si un emplacement horizontal `H` et un emplacement vertical `V` se croisent, la lettre à l'intersection est la même dans les deux mots assignés.

# 3. Choix de l'Approche : Programmation par Contraintes (PPC)

La PPC est particulièrement adapté pour plusieurs raisons :
*   **Modélisation Naturelle :** Permet de décrire le problème en termes de variables (quel mot pour quel emplacement ?), de domaines (quels sont les mots possibles ?) et de contraintes (les lettres aux intersections doivent correspondre).
*   **Solveurs Efficaces :** CP-SAT intègre des algorithmes pour explorer l'espace des solutions et trouver une assignation valide (si elle existe) rapidement pour des tailles de problèmes raisonnables.
 *   **Flexibilité :** Facilité d'ajout de nouvelles contraintes.

# 4. Modélisation du Problème

 ### 4.1. Représentation de la Grille

La grille est représentée par une liste de listes Python. '.' représente une case blanche, '#' une case noire.
Nous utilisons une fonction pour générer une grille aléatoire simple.

In [2]:
import sys
import time
from ortools.sat.python import cp_model
import re
import random
from openai import OpenAI
import os

def generate_table(rows = 6, columns = 6):
    """Génère une grille aléatoire simple."""
    noir_colonne_precedente = True # eviter 2 lignes noirs d'affile
    res = []
    for r in range(rows):
        row = ["."] * columns
        has_black = False
        if random.random() < 0.85 or noir_colonne_precedente:
            num_noires = random.randint(1, max(1, columns // 3))
            for _ in range(num_noires):
                if columns > 2:
                    noir = random.randint(1, columns - 2)
                else:
                    noir = random.randint(0, columns - 1)
                if row[noir] == '.':
                    row[noir] = "#"
                    has_black = True

        noir_colonne_precedente = has_black
        res.append(row)

    total_blanches = sum(row.count('.') for row in res)
    if total_blanches == rows * columns:
        rand_r, rand_c = random.randint(0, rows-1), random.randint(0, columns-1)
        res[rand_r][rand_c] = '#'

    return res

### Exemple de génération et affichage:


In [3]:
ROWS = 6
COLUMNS = 6
LA_GRILLE = generate_table(ROWS, COLUMNS)

print("Grille Générée:")
for r in LA_GRILLE:
    print(" ".join(r))

Grille Générée:
. # . # . .
. . . # . .
. # . . # .
. . . . # .
. . . . # .
. . . # . .



### 4.2. Identification des Emplacements (Slots)

Un emplacement ("slot") est une séquence continue de cases blanches (horizontale ou verticale) d'une longueur minimale donnée. Nous parcourons la grille pour les identifier.

### 4.3. Identification des Croisements

Un croisement se produit lorsqu'un slot horizontal et un slot vertical partagent la même case blanche. Nous devons identifier ces points et les indices correspondants dans chaque mot.

La fonction `trouver_emplacements_et_croisements` gère ces deux étapes.

In [4]:
longueur_min_mot = 2

def trouver_emplacements_et_croisements(grille):
    """Identifie les emplacements et les croisements dans la grille."""
    hauteur = len(grille)
    largeur = len(grille[0])
    liste_emplacements = []
    compteur_emplacements = 0
    longueurs_requises = set()

    for r in range(hauteur):
        c = 0
        while c < largeur:
            if grille[r][c] == '.':
                col_debut_mot = c
                longueur_mot = 0
                while c < largeur and grille[r][c] == '.':
                    longueur_mot += 1
                    c += 1
                if longueur_mot >= longueur_min_mot:
                    nouvel_emp = (compteur_emplacements, "H", r, col_debut_mot, longueur_mot)
                    liste_emplacements.append(nouvel_emp)
                    longueurs_requises.add(longueur_mot)
                    compteur_emplacements += 1
            else:
                c += 1

    for c in range(largeur):
        r = 0
        while r < hauteur:
            if grille[r][c] == '.':
                ligne_debut_mot = r
                longueur_mot = 0
                while r < hauteur and grille[r][c] == '.':
                    longueur_mot += 1
                    r += 1
                if longueur_mot >= longueur_min_mot:
                    nouvel_emp = (compteur_emplacements, 'V', ligne_debut_mot, c, longueur_mot)
                    liste_emplacements.append(nouvel_emp)
                    longueurs_requises.add(longueur_mot)
                    compteur_emplacements += 1
            else:
                r += 1

    liste_croisements = []
    map_emplacements = {emp[0]: emp for emp in liste_emplacements}

    map_cellules = {}
    for emp_id, direction, r_start, c_start, length in liste_emplacements:
        for i in range(length):
            if direction == 'H':
                cell = (r_start, c_start + i)
            else: # 'V'
                cell = (r_start + i, c_start)

            if cell not in map_cellules:
                map_cellules[cell] = []
            map_cellules[cell].append(emp_id)

    paires_traitees = set()
    for cell, ids_occupants in map_cellules.items():
        if len(ids_occupants) > 1:
             id_h = -1
             id_v = -1
             for un_id in ids_occupants:
                 if map_emplacements[un_id][1] == 'H':
                     id_h = un_id
                 else:
                     id_v = un_id

             if id_h != -1 and id_v != -1:
                 paire = tuple(sorted((id_h, id_v)))

                 if paire not in paires_traitees:
                    emp_h = map_emplacements[id_h]
                    emp_v = map_emplacements[id_v]
                    index_dans_mot_h = cell[1] - emp_h[3]
                    index_dans_mot_v = cell[0] - emp_v[2]

                    details_croisement = {
                        'h_slot_id': id_h,
                        'v_slot_id': id_v,
                        'h_char_index': index_dans_mot_h,
                        'v_char_index': index_dans_mot_v
                    }
                    liste_croisements.append(details_croisement)
                    paires_traitees.add(paire)

    return liste_emplacements, liste_croisements, longueurs_requises

### Exemple d'exécution:


In [5]:
les_emplacements, les_croisements, les_longueurs_requises = trouver_emplacements_et_croisements(LA_GRILLE)

print(f"Nombre d'emplacements trouvés: {len(les_emplacements)}")
print("Emplacements (id, dir, row, col, len):", les_emplacements)
print(f"Nombre de croisements trouvés: {len(les_croisements)}")
print("Croisements:", les_croisements)
print(f"Longueurs de mots requises: {les_longueurs_requises}")

Nombre d'emplacements trouvés: 14
Emplacements (id, dir, row, col, len): [(0, 'H', 0, 4, 2), (1, 'H', 1, 0, 3), (2, 'H', 1, 4, 2), (3, 'H', 2, 2, 2), (4, 'H', 3, 0, 4), (5, 'H', 4, 0, 4), (6, 'H', 5, 0, 3), (7, 'H', 5, 4, 2), (8, 'V', 0, 0, 6), (9, 'V', 3, 1, 3), (10, 'V', 0, 2, 6), (11, 'V', 2, 3, 3), (12, 'V', 0, 4, 2), (13, 'V', 0, 5, 6)]
Nombre de croisements trouvés: 20
Croisements: [{'h_slot_id': 0, 'v_slot_id': 12, 'h_char_index': 0, 'v_char_index': 0}, {'h_slot_id': 0, 'v_slot_id': 13, 'h_char_index': 1, 'v_char_index': 0}, {'h_slot_id': 1, 'v_slot_id': 8, 'h_char_index': 0, 'v_char_index': 1}, {'h_slot_id': 1, 'v_slot_id': 10, 'h_char_index': 2, 'v_char_index': 1}, {'h_slot_id': 2, 'v_slot_id': 12, 'h_char_index': 0, 'v_char_index': 1}, {'h_slot_id': 2, 'v_slot_id': 13, 'h_char_index': 1, 'v_char_index': 1}, {'h_slot_id': 3, 'v_slot_id': 10, 'h_char_index': 0, 'v_char_index': 2}, {'h_slot_id': 3, 'v_slot_id': 11, 'h_char_index': 1, 'v_char_index': 0}, {'h_slot_id': 4, 'v_slot_

### 4.4. Gestion du Dictionnaire

Nous chargeons les mots depuis un fichier texte. Pour optimiser, nous ne gardons en mémoire que les mots dont la longueur correspond à l'une des longueurs requises par les emplacements de la grille. Les mots sont stockés dans un dictionnaire où les clés sont les longueurs.

En entrée nous avons un fichier ```dictionnaire.txt``` qui contient une grande liste de mot

In [6]:
dictionnaire_mots_fichier = 'dictionnaire.txt'

def charger_dico(nom_fichier, longueurs_requises):
    """Charge les mots du dictionnaire ayant les longueurs spécifiées."""
    mots_par_longueur = {lg: [] for lg in longueurs_requises}
    try:
        with open(nom_fichier, 'r', encoding='utf-8') as fichier:
            for ligne in fichier:
                mot = ligne.strip().upper()
                if mot.isalpha():
                    lg = len(mot)
                    if lg in longueurs_requises:
                        mots_par_longueur[lg].append(mot)
    except FileNotFoundError:
        print(f"ERREUR: Le fichier dictionnaire '{nom_fichier}' est introuvable.")
        return None

    for lg in longueurs_requises:
        if not mots_par_longueur[lg]:
             print(f"AVERTISSEMENT: Aucun mot de longueur {lg} trouvé dans le dictionnaire.")

    return mots_par_longueur

dico_mots_par_longueur = charger_dico(dictionnaire_mots_fichier, les_longueurs_requises)

if dico_mots_par_longueur is not None:
    print("Dictionnaire chargé.")
    for lg, mots in dico_mots_par_longueur.items():
        print(f" - Longueur {lg}: {len(mots)} mots")

    mots_possibles_par_emplacement = {}
    probleme_valide = True
    if les_emplacements:
        for emp_id, _, _, _, longueur_voulue in les_emplacements:
            candidats = dico_mots_par_longueur.get(longueur_voulue, [])
            if not candidats:
                print(f"ERREUR: Aucun mot de longueur {longueur_voulue} disponible pour l'emplacement {emp_id}. La grille ne peut pas être remplie.")
                probleme_valide = False
            mots_possibles_par_emplacement[emp_id] = candidats
    else:
        print("Aucun emplacement trouvé dans la grille.")
        probleme_valide = False

else:
    probleme_valide = False

Dictionnaire chargé.
 - Longueur 2: 58 mots
 - Longueur 3: 335 mots
 - Longueur 4: 992 mots
 - Longueur 6: 3146 mots


### 4.5. Formulation CSP (Variables, Domaines, Contraintes)

 *   **Variables :** Une variable entière par emplacement (`slot_id`).
 *   **Domaines :** Pour une variable associée à l'emplacement `i`, son domaine est `[0, 1, ..., k-1]`, où `k` est le nombre de mots candidats pour cet emplacement. Chaque entier représente l'index d'un mot dans la liste des mots possibles.
 *   **Contraintes :** Pour chaque croisement entre un slot horizontal `H` (variable `var_H`) et un slot vertical `V` (variable `var_V`), nous ajoutons une contrainte. Si le croisement concerne la `h_idx`-ième lettre de `H` et la `v_idx`-ième lettre de `V`, la contrainte impose que `mot_H[h_idx] == mot_V[v_idx]`, où `mot_H` est le mot choisi pour `H` (correspondant à la valeur de `var_H`) et `mot_V` est le mot choisi pour `V` (correspondant à la valeur de `var_V`).
     *   Avec CP-SAT, ceci est efficacement modélisé par `model.AddAllowedAssignments([var_H, var_V], liste_paires_compatibles)`. `liste_paires_compatibles` contient tous les couples `(index_mot_H, index_mot_V)` tels que les mots correspondants respectent la contrainte d'égalité de la lettre au croisement.

## 5. Implémentation avec OR-Tools CP-SAT

Nous allons maintenant construire le modèle CP-SAT, le résoudre et afficher le résultat.

Fonction pour afficher la grille (vide ou remplie):

In [7]:
def afficher_grille(structure_grille, la_solution=None, les_emplacements_map=None):
    """Affiche la grille, potentiellement remplie avec la solution."""
    hauteur = len(structure_grille)
    largeur = len(structure_grille[0])
    grille_affichee = [list(row) for row in structure_grille]

    if la_solution is not None and les_emplacements_map is not None:
         for id_emp, mot_choisi in la_solution.items():
             _, direction, r_start, c_start, _ = les_emplacements_map[id_emp]
             for index_lettre, lettre in enumerate(mot_choisi):
                 if direction == "H":
                     colonne_case = c_start + index_lettre
                     ligne_case = r_start
                 else: # 'V'
                     colonne_case = c_start
                     ligne_case = r_start + index_lettre
                 if 0 <= ligne_case < hauteur and 0 <= colonne_case < largeur:
                     if grille_affichee[ligne_case][colonne_case] == '.':
                         grille_affichee[ligne_case][colonne_case] = lettre
                 else:
                     print(f"Attention: Coordonnée hors grille ({ligne_case}, {colonne_case}) pour slot {id_emp}")


    print("-" * (largeur * 2 - 1))
    for r in range(hauteur):
        print(" ".join(grille_affichee[r]))
    print("-" * (largeur * 2 - 1))

## 6. Génération des définitions avec openAI

Une fois que nous avons la grille avec les mots utilisés, il faut pouvoir donner une définition à chaque mot. Nous avons décider d'utiliser un modèle LLM de OpenAI pour le faire automatiquement. Il nous faut simplement appeler le modèle avec tous les mots et un prompt de base, puis récupérer ces définitions et les afficher correctement.

*(Ne pas oublier de mettre la clé openAI)* 

In [ ]:
OPENAI_API_KEY = "" #Mettre clé OpenAI ici

if OPENAI_API_KEY == "":
    raise Exception("Clé OpenAI vide")
client = OpenAI(
  api_key=OPENAI_API_KEY,
)
base_prompt = ("Je souhaite créer des grilles de mots croisés, et j'ai une liste de mots français pour la grille mais je n'ai pas les"
          " définitions des mots. Donne moi une définition dans le cadre d'un mot croisé pour chaque mot de cette liste de mot, "
          "sans numéroter les lignes et en mettant une définition par ligne sans le mot, avec rien d'autre que la définition, sans sauter de lignes et sans tirets."
          "Voici la liste: \n")

def create_definitions(dictionary):
    words = '\n'.join(dictionary.values())

    completion = client.chat.completions.create(
        model="gpt-4o", 
        messages=[{"role": "user", "content": base_prompt + words}]
    )

    result = completion.choices[0].message.content
    defs = result.split('\n')
    return defs

def afficher_definitions(solution):
  defs = create_definitions(solution)

  mots_horizontal = {}
  mots_vertical = {}  
  for i in range(ROWS):
      mots_horizontal[i] = []
  for i in range(COLUMNS):
      mots_vertical[i] = []
  
  i = 0
  for e in les_emplacements:
      if e[1] == 'H':
          mots_horizontal[e[2]].append(defs[i])
      else:
          mots_vertical[e[3]].append(defs[i])
      i += 1
      
  print("\nHORIZONTAL:")
  for k, v in mots_horizontal.items():
      print(k + 1, '-', " // ".join(v))
  print("VERTICAL:")
  for k, v in mots_vertical.items():
      print(k + 1, '-', " // ".join(v))

# --- Construction et Résolution du Modèle ---


In [14]:
solution_trouvee = None

if probleme_valide and dico_mots_par_longueur is not None and les_emplacements:

    modele = cp_model.CpModel()

    vars_emplacement = {}
    map_emplacements_details = {emp[0]: emp for emp in les_emplacements}

    for emp_id in mots_possibles_par_emplacement:
        nb_candidats = len(mots_possibles_par_emplacement[emp_id])
        if nb_candidats > 0:
            nom_variable = f'slot_{emp_id}'
            vars_emplacement[emp_id] = modele.NewIntVar(0, nb_candidats - 1, nom_variable)
        else:
            print(f"ERREUR Interne: Emplacement {emp_id} n'a aucun mot candidat au moment de créer la variable.")
            probleme_valide = False
            break

    if probleme_valide:
        print(f"Ajout de {len(les_croisements)} contraintes de croisement...")
        nb_contraintes_ajoutees = 0
        for croisement in les_croisements:
            id_h = croisement['h_slot_id']
            id_v = croisement['v_slot_id']
            index_lettre_h = croisement['h_char_index']
            index_lettre_v = croisement['v_char_index']

            var_h = vars_emplacement[id_h]
            var_v = vars_emplacement[id_v]
            mots_h = mots_possibles_par_emplacement[id_h]
            mots_v = mots_possibles_par_emplacement[id_v]

            affectations_permises = []
            for index_candidat_h, mot_h in enumerate(mots_h):
                for index_candidat_v, mot_v in enumerate(mots_v):
                    if mot_h[index_lettre_h] == mot_v[index_lettre_v]:
                        affectations_permises.append( (index_candidat_h, index_candidat_v) )

            if affectations_permises:
                modele.AddAllowedAssignments([var_h, var_v], affectations_permises)
                nb_contraintes_ajoutees += 1
            else:
                print(f"INFO: Aucune paire de mots compatible trouvée pour le croisement H={id_h}, V={id_v}. Le problème est probablement infaisable.")
                modele.AddBoolOr([])

        print(f"{nb_contraintes_ajoutees} contraintes AddAllowedAssignments ajoutées.")

        if probleme_valide:
            print("\nLancement du solveur CP-SAT...")
            solveur = cp_model.CpSolver()
            solveur.parameters.num_search_workers = 4
            start_time = time.time()
            statut = solveur.Solve(modele)
            end_time = time.time()
            print(f"Résolution terminée en {end_time - start_time:.2f} secondes.")

            if statut == cp_model.OPTIMAL or statut == cp_model.FEASIBLE:
                print("\nSolution trouvée !")
                solution_trouvee = {}
                for id_emp, variable in vars_emplacement.items():
                    index_mot_retenu = solveur.Value(variable)
                    mot_retenu = mots_possibles_par_emplacement[id_emp][index_mot_retenu]
                    solution_trouvee[id_emp] = mot_retenu

                print("\nGrille Remplie :")
                afficher_grille(LA_GRILLE, solution_trouvee, map_emplacements_details)
                afficher_definitions(solution_trouvee)

            elif statut == cp_model.INFEASIBLE:
                print("\nInfaisable")
            elif statut == cp_model.MODEL_INVALID:
                 print("\nModele invalide")
            else:
                print(f"\nErreur : {statut}")

elif not les_emplacements and dico_mots_par_longueur is not None:
    print("\nPas d'emplacements à remplir")
    afficher_grille(LA_GRILLE)
elif not probleme_valide:
    print("\nPas valide")
    print("Affichage de la grille initiale :")
    afficher_grille(LA_GRILLE)

Ajout de 20 contraintes de croisement...
20 contraintes AddAllowedAssignments ajoutées.

Lancement du solveur CP-SAT...
Résolution terminée en 2.81 secondes.

Solution trouvée !

Grille Remplie :
-----------
A # B # A S
C S A # H O
H # L U # I
E M I S # E
V I S E # N
E T E # E T
-----------

HORIZONTAL:
1 - Un avion sans pilote  
2 - Confédération des syndicats autonomes   // Expression d'hésitation  
3 - Lu à haute voix  
4 - Diffusés par transmission  
5 - Cible un objectif  
6 - Saison chaude   // Conjonction de coordination  
VERTICAL:
1 - Complété ou terminé  
2 - Il placarda des affiches  
3 - Indique un repère  
4 - Fin d'une carrière parfois  
5 - Expression de surprise  
6 - Pour qu'ils le soient


# 7. Expérimentations et Évolution (Discussion)

 *   **Génération de Grille :** La fonction `generate_table` est simple. La qualité de la grille initiale influence la possibilité de trouver une solution. Le problème avec cette fonction est que la position des cases noir est généré de façon plus ou moins aléatoire et donc il y a des cases blanches qui peuvent être isolées des autres cases blanches, cela peut egalement entrainer la formation de mots de une lettre seulement ce qui n'est pas souhaitable.
*   **Taille du Dictionnaire :** Un dictionnaire plus grand offre plus de possibilités mais augmente considérablement le temps de calcul des paires compatibles (`affectations_permises`) et la complexité pour le solveur. Le filtrage par longueur est crucial.
*   **Performance :** Pour des grilles de taille modeste (6x6, 8x8) et un dictionnaire raisonnable, CP-SAT est très rapide. Pour des grilles plus grandes (15x15+) et/ou des dictionnaires très volumineux, le temps de construction du modèle et le temps de résolution peuvent devenir énormes.

# 8. Résultats Obtenus

Le programme est capable de :
1.  Générer une structure de grille aléatoire.
2.  Identifier les emplacements et les croisements.
3.  Charger et filtrer un dictionnaire.
4.  Modéliser le problème en CSP avec OR-Tools.
5.  Trouver une solution (grille remplie) si elle existe.
6.  Afficher la grille vide initiale et la grille remplie (si solution trouvée).

## 9. Conclusion et Perspectives

Ce notebook démontre l'application réussie de la programmation par contraintes avec OR-Tools CP-SAT pour la génération de mots-croisés. Le modèle capture bien les dépendances entre les mots.